**General**
For ease of use, we recommend downloading the original Real-ESRGAN reposetory (https://github.com/xinntao/Real-ESRGAN), and uploading the folder to a google colab.
In any case, in order to run this notebook it is required to put it int the Real-ESRGAN folder.

The notebook is comprosed of 2 sections, training and inference. If only one is required, skip the irrelevent part.

In [2]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# !cp -r /content/drive/MyDrive/Real-ESRGAN-master/* .

In [14]:
import os
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# facexlib and gfpgan are for face enhancement
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop
!pip install piqa

^C
ERROR: Operation cancelled by user
fatal: detected dubious ownership in repository at '/home/nabila/projects/lume/super_resolution/Real-ESRGAN/Real-ESRGAN'
To add an exception for this directory, call:

	git config --global --add safe.directory /home/nabila/projects/lume/super_resolution/Real-ESRGAN/Real-ESRGAN
/home/nabila/projects/lume/super_resolution/Real-ESRGAN/Real-ESRGAN/.venv/lib/python3.10/site-packages/setuptools/__init__.py:81: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/home/nabila/projects/lume/super_resolution/Real-ESRGAN/Real-ESRGAN/.venv/lib/python3.10/sit

**Training**

You will need to download via the code below the architecture dataset from kaggle (https://www.kaggle.com/datasets/tompaulat/modernarchitecture). In order to do so, you must have a kaggle account and download a kaggle.json file. After you have downloaded it or if you have one available, upload it with the cell below.

In [4]:
# from google.colab import files
# files.upload() #this will prompt you to upload the kaggle.json

# This cell will install kaggle support and download the modernarchitecture dataset and unpack it.
# !ls -lha kaggle.json
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 /root/.kaggle/kaggle.json
# !kaggle datasets download -d tompaulat/modernarchitecture
# !mkdir datasets
# !unzip modernarchitecture.zip -d datasets/modernarchitecture

# Here we will unzip the dataset into a folder to access the images
# !mkdir datasets
# !unzip modernarchitecture.zip -d datasets/modernarchitecture

# We shall divide the images into 3 folders/datasets: train, test and validation.
'''
!mkdir f"{dataset_path}"/validation/
!mkdir f"{input_path}"/validation/hq/
!mkdir f"{input_path}"/test/
!mkdir f"{input_path}"/test/hq/
!mkdir f"{input_path}"/train/
'''

# !mv -v datasets/modernarchitecture/Appartments/Appartments/X-Z/*.jpg datasets/modernarchitecture/validation/hq/
# !mv -v datasets/modernarchitecture/Public_Buildings/Public_Buildings/X-Z/*.jpg datasets/modernarchitecture/validation/hq/

# !mv -v datasets/modernarchitecture/Appartments/Appartments/F-H/F* datasets/modernarchitecture/test/hq/
# !mv -v datasets/modernarchitecture/Appartments/Appartments/F-H/* datasets/modernarchitecture/test/hq/
# !mv -v datasets/modernarchitecture/Public_Buildings/Public_Buildings/F-H/*.jpg datasets/modernarchitecture/test/hq/

# !mv -v datasets/modernarchitecture/Appartments/Appartments/R-T/R*.jpg datasets/modernarchitecture/train/
# !for d in datasets/modernarchitecture/Appartments/Appartments/*; do mv -v $d/*.jpg datasets/modernarchitecture/train/; done
# !for d in datasets/modernarchitecture/Public_Buildings/Public_Buildings/*; do mv -v $d/*.jpg datasets/modernarchitecture/train/; done

# !mv "datasets/modernarchitecture/Public Buildings/"  datasets/modernarchitecture/Public_Buildings
# !mv "datasets/modernarchitecture/Public_Buildings/Public Buildings"  datasets/modernarchitecture/Public_Buildings/Public_Buildings

# The original file names have spaces in them. This causes proble,s to future scripts and functions, So we will replace the spaces with lower dash "_".

'''!for f in datasets/modernarchitecture/train/*; do mv "$f" `echo $f | tr ' ' '_'`; done
!for f in datasets/modernarchitecture/validation/hq/*; do mv "$f" `echo $f | tr ' ' '_'`; done
!for f in datasets/modernarchitecture/test/hq/*; do mv "$f" `echo $f | tr ' ' '_'`; done'''

We have low quality (LQ) images at the input (train and test) and high quality images (HQ) at the output. HQ images are used as ground-truth.
The follow steps are in 'github.com/xinntao/Real-ESRGAN/blob/master/docs/Training.md'

The scheme path to the directorys are:

- datasets/
    - meta_info_train.txt
    - meta_info_pair.txt
    - input
    - output
    - test
        - lq
            - 00
                - 00_image.png
            - 01
            ( ... )
        - hq
            - 00
                - 00_image.png
            - 01
            ( ... )
    - train
        - 05_image.png
          ( ... )
    - validation
        - lq
            - 03
                - 03_image.png
            - 04
            ( ... )
        - hq
            - 03
                - 03_image.png
            - 04
            ( ... )

In [ ]:
!cp -v /mnt/d/Dev/Projects/Lume/super_resolution/datasets/input/splited_images/00/*.png /mnt/d/Dev/Projects/Lume/super_resolution/datasets/validation/lq/ # (9k img)
!cp -v /mnt/d/Dev/Projects/Lume/super_resolution/datasets/output/original_imgs/00/*.png /mnt/d/Dev/Projects/Lume/super_resolution/datasets/validation/hq/ # (9k img)

# test
!cp -v /mnt/d/Dev/Projects/Lume/super_resolution/datasets/input/splited_images/01/*.png /mnt/d/Dev/Projects/Lume/super_resolution/datasets/test/lq/ # 11k

# train
!cp -v /mnt/d/Dev/Projects/Lume/super_resolution/datasets/input/splited_images/02/*.png /mnt/d/Dev/Projects/Lume/super_resolution/datasets/train/ # 20k
!cp -v /mnt/d/Dev/Projects/Lume/super_resolution/datasets/input/splited_images/03/*.png /mnt/d/Dev/Projects/Lume/super_resolution/datasets/train/ # 22k img
find /mnt/d/Dev/Projects/Lume/super_resolution/datasets/input/splited_images/02/ -name "*.png" -type f -print0 | xargs -0 -I {} cp -v {} /mnt/d/Dev/Projects/Lume/super_resolution/datasets/train/

In [36]:
dataset_path = "/mnt/d/Dev/Projects/Lume/super_resolution/datasets/"
input_path = os.path.join(f"{dataset_path}","input","splited_images") # LQ
output_path = os.path.join(f"{dataset_path}","output","original_imgs") # HQ

# Directories for train, test, and validation splits
train_dir = os.path.join(f"{dataset_path}", "train")
test_dir_hq = os.path.join(f"{dataset_path}","test" ,"hq")
test_dir_lq = os.path.join(f"{dataset_path}","test" ,"lq")
val_dir_hq = os.path.join(f"{dataset_path}", "validation", "hq")
val_dir_lq = os.path.join(f"{dataset_path}", "validation", "lq")

# Createthe directorys if they not exists
if not os.path.exists(train_dir):
    os.makedirs(train_dir, exist_ok=True)
if not os.path.exists(test_dir):
    os.makedirs(test_dir, exist_ok=True)
if not os.path.exists(val_dir):
    os.makedirs(val_dir, exist_ok=True)

Rename the files in the output folder so that they have the same file name as the input folder

In [ ]:
import os

# rename validation files name
def rename_files(directory):
    # List all files in the specified directory
    files = os.listdir(directory)
    print(f"path {directory}")
    for file in files:
        # Check if the file name contains "_original"
        print(f"checking file: {file}")
        if '_original' in file:
            # Generate the new file name by removing "_original"
            new_name = file.replace('_original', '')

            # Rename the file
            os.rename(os.path.join(directory, file), os.path.join(directory, new_name))


# Rename the files
rename_files(val_dir_hq)
rename_files(val_dir_lq)

if you want to organize the dataset directories to the format: datasets > input > 00 > 00_image.png

In [ ]:
def organize_files(base_directory):
    # List all files in the base directory
    files = os.listdir(base_directory)
    print(f"Organizando {base_directory}...")

    for file in files:
        # Skip directories, we only want to move files
        if os.path.isfile(os.path.join(base_directory, file)):
            # Extract the prefix (first two characters) from the filename
            prefix = file[:2]
            print(f"Ordem {prefix}...")

            # Create a new directory with the prefix if it doesn't already exist
            new_directory = os.path.join(base_directory, prefix)
            if not os.path.exists(new_directory):
                os.makedirs(new_directory)

            # Move the file to the new directory
            shutil.move(os.path.join(base_directory, file), os.path.join(new_directory, file))

# Organize the files
organize_files(input_path)
organize_files(output_path)

If you wanna organize the directory with all the images on train, test and validation:

In [13]:
from sklearn.model_selection import train_test_split

file_list = os.listdir(input_path)

# Split the dataset into feature and target sets
train_files, test_files = train_test_split(file_list, test_size=0.2, random_state=42)
train_files, val_files = train_test_split(train_files, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

In [ ]:
import os
import shutil

# Function to move files
def copy_files(files, source, destination):
    print(f"Copy from {source} to {destination}")
    for file in files:
        # Construct full file path
        src_path = os.path.join(source, file)
        dst_path = os.path.join(destination, file)
        # Move the file
        shutil.copy(src_path, dst_path)

# Move the files according to the splits
copy_files(train_files, input_path, train_dir)
copy_files(test_files, input_path, test_dir)
copy_files(val_files, input_path, val_dir)

print("Number of files in train set:", len(os.listdir(train_dir)))
print("Number of files in test set:", len(os.listdir(test_dir)))
print("Number of files in validation set:", len(os.listdir(val_dir)))

To use the validation, we must create lower resultion versions of the same images.

In HQ images we have images with dimensions 64x1024 and in LQ 16x1024. To use this model, we must have the same image dimensions between input and output, so we resize the LQ images by interpolating their lines.

In [ ]:
import numpy as np
import cv2

def interpolate_lines(input_image):
    # Get the dimensions of the input image
    input_height, input_width = input_image.shape

    # Calculate the scale factor for interpolation
    scale_factor = 64 // input_height
    print('scale_factor = ',scale_factor)

    # Create an empty output image with the desired size
    output_height = 64
    output_width = input_width
    output_image = np.zeros((output_height, output_width), dtype=np.uint8)

    # Interpolate lines between the input lines
    for i in range(input_height - 1):
        start_line = input_image[i]
        end_line = input_image[i + 1]

        # Interpolate between the start and end lines
        interpolated_lines = np.linspace(start_line, end_line, scale_factor + 1, axis=0)

        # Copy the interpolated lines to the output image
        output_image[i * scale_factor : (i + 1) * scale_factor] = interpolated_lines[:-1]

    # Copy the last line of the input image to the remaining lines of the output image
    output_image[(input_height - 1) * scale_factor :] = input_image[-1]

    return output_image

input_path_00 = os.path.join(f"{dataset_path}","input","splited_images", "00")

for file in os.listdir(input_path_00):
    filePath = os.path.join(input_path_00, file)
    print(f"Interpolating {filePath}...")
    input_image = cv2.imread(filePath, cv2.IMREAD_GRAYSCALE)
    output_image = interpolate_lines(input_image)
    cv2.imwrite(filePath, output_image)

In [35]:
# !mkdir /mnt/d/Dev/Projects/Lume/super_resolution/datasets/validation/lq/
# from PIL import Image
# import glob
# for filename in glob.glob('/mnt/d/Dev/Projects/Lume/super_resolution/datasets/validation/hq/*'):
#   head, tail = os.path.split(filename)
#   image_golden = Image.open(filename)
#   image_downscale = image_golden.resize((int(image_golden.size[0]/4),int(image_golden.size[1]/4)))
#   image_downscale.save("/mnt/d/Dev/Projects/Lume/super_resolution/datasets/validation/lq/" + tail)


## START HERE

As we already have both directorys (input LQ and output HQ), we'll finetuning with our own paired data. For this, we will use only the input and output directories and the config file meta_train_pair.txt.

The original Real-ESRGAN repo requires to create a meta info file for the train dataset. This cell will run a script to generate exactly that.
In .txt file, in our example, is necessery to rename the paths to replace "../../../../../../.." to ""

In [ ]:
# !python scripts/generate_meta_info.py --input datasets/modernarchitecture/train/ --root ./ --meta_info datasets/modernarchitecture/meta_info_train.txt
!python scripts/generate_meta_info.py --input /mnt/d/Dev/Projects/Lume/super_resolution/datasets/train/ --root ./ --meta_info /mnt/d/Dev/Projects/Lume/super_resolution/datasets/meta_info_train.txt

In [ ]:
# if ypu want Train the model
!python realesrgan/train.py --input /mnt/d/Dev/Projects/Lume/super_resolution/datasets/output/original_imgs/00 /mnt/d/Dev/Projects/Lume/super_resolution/datasets/input/splited_images/00 --meta_info /mnt/d/Dev/Projects/Lume/super_resolution/datasets/meta_info_pair.txt

In [ ]:
# if you want to Finetuning with you own paired data
!python realesrgan/train_finetune_architecture.py -opt options/finetune_realesrgan_x4plus_pairdata.yml --auto_resume

In [6]:
# !rm datasets/modernarchitecture/train/meta_info_train.txt
!rm /mnt/d/Dev/Projects/Lume/super_resolution/datasets/train/meta_info_train.txt

rm: cannot remove '/mnt/d/Dev/Projects/Lume/super_resolution/datasets/train/meta_info_train.txt': No such file or directory


Here we shall download the original Real-ESRGAN weights. If you are going to work on different weights, you can skip this part.

In [ ]:
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.3/RealESRGAN_x4plus_netD.pth -P experiments/pretrained_models

And finally, run this cell in order to strat the training process.

In [ ]:
!python realesrgan/train_finetune_architecture.py -opt options/finetune_realesrgan_x4plus_pairdata.yml --auto_resume

We will also need to copy the new weights to the wegihts folder from the experiment folder

In [3]:
# !cp experiments/finetune_RealESRGANx4plus_400k/models/net_g_latest.pth weights/net_g_latest.pth
!cp /experiments/finetune_RealESRGANx4plus_400k_pairdata/models/net_g_latest.pth weights/net_g_latest.pth

cp: cannot stat '/experiments/finetune_RealESRGANx4plus_400k_pairdata/models/net_g_latest.pth': No such file or directory


Now we shall test the trained model. Similiar to the validation set, we first must create the lower resoultion versions of the test set.

**Inference and Test**

In [ ]:
# test
!cp -v /mnt/d/Dev/Projects/Lume/super_resolution/datasets/output/original_imgs/01/*.png /mnt/d/Dev/Projects/Lume/super_resolution/datasets/test/hq/ # 11k

In [9]:
import os
import shutil
import random

dataset_path = "/mnt/d/Dev/Projects/Lume/super_resolution/datasets/"
source_path = os.path.join(f"{dataset_path}","input","splited_images", "01") # LQ
validation_path = os.path.join(f"{source_path}","validation")
test_path = os.path.join(f"{source_path}","test")

# Create the directories if they don't exist
if not os.path.exists(validation_path):
    os.makedirs(validation_path, exist_ok=True)
if not os.path.exists(test_path):
    os.makedirs(test_path, exist_ok=True)

print(source_path)
print(validation_path)
print(test_path)

# Get a list of all .png files in the source path
file_list = [file for file in os.listdir(source_path) if file.endswith(".png")]

# Move each file to the validation path
for file in file_list:
    src_file = os.path.join(source_path, file)
    dst_file = os.path.join(validation_path, file)
    shutil.move(src_file, dst_file)







/mnt/d/Dev/Projects/Lume/super_resolution/datasets/input/splited_images/01
/mnt/d/Dev/Projects/Lume/super_resolution/datasets/input/splited_images/01/validation
/mnt/d/Dev/Projects/Lume/super_resolution/datasets/input/splited_images/01/test


In [ ]:
import os

# rename validation files name
def rename_files(directory):
    # List all files in the specified directory
    files = os.listdir(directory)
    print(f"path {directory}")
    for file in files:
        # Check if the file name contains "_original"
        print(f"checking file: {file}")
        if '_original' in file:
            # Generate the new file name by removing "_original"
            new_name = file.replace('_original', '')

            # Rename the file
            os.rename(os.path.join(directory, file), os.path.join(directory, new_name))


# Rename the files
rename_files(test_dir_hq)
# rename_files(val_dir_lq)

In [ ]:
!mkdir inputs_test/
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import glob
for filename in glob.glob('datasets/modernarchitecture/test/hq/*'):
  image_golden = Image.open(filename)
  image_downscale = image_golden.resize((int(image_golden.size[0]/2),int(image_golden.size[1]/2)))
  head, tail = os.path.split(filename)
  image_downscale.save("inputs_test/downscale_" + tail)

mkdir: cannot create directory ‘inputs_test/’: File exists


Finally, we can run the inference script in order ro get the test results

In [13]:
# !python inference_realesrgan_architecture.py -n net_g_latest.pth -i inputs_test --outscale 2 --output new_model_results
!python inference_realesrgan.py -n net_g_latest.pth -i /mnt/d/Dev/Projects/Lume/super_resolution/datasets/input/splited_images/01/test --outscale 4 --output new_model_results

net_g_latest
weights/net_g_latest.pth
4
Traceback (most recent call last):
  File "/home/nabila/projects/lume/super_resolution/Real-ESRGAN/Real-ESRGAN/inference_realesrgan.py", line 170, in <module>
    main()
  File "/home/nabila/projects/lume/super_resolution/Real-ESRGAN/Real-ESRGAN/inference_realesrgan.py", line 115, in main
    model=model,
UnboundLocalError: local variable 'model' referenced before assignment


Or we can infer with previosly fine tuned model

In [16]:
# !python inference_realesrgan.py -n net_g_architecture.pth -i /mnt/d/Dev/Projects/Lume/super_resolution/datasets/input/splited_images/01/test --outscale 4 --output /mnt/d/Dev/Projects/Lume/super_resolution/datasets/input/splited_images/01/results
!python inference_realesrgan.py -n net_g_architecture.pth -i /mnt/d/Dev/Projects/Lume/super_resolution/datasets/input/splited_images/01/LR --outscale 4 --output /mnt/d/Dev/Projects/Lume/super_resolution/datasets/input/splited_images/01/LR_results

net_g_architecture
weights/net_g_architecture.pth
4
Testing 0 1_000097
Testing 1 1_000119
Testing 2 2_000097
Testing 3 2_000119
Testing 4 3_000097
Testing 5 3_000119
Testing 6 4_000097
Testing 7 4_000119


If there is a need, you could run the cell below in order to use the original Real-ESRGAN weights, and to test them

In [ ]:
!python inference_realesrgan_architecture.py -n RealESRGAN_x4plus -i inputs_test --outscale 2 --output golden_results

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
Testing 0 downscale_F176_VOX_arquitectura_Beam_
Testing 1 downscale_F176_VOX_arquitectura_Door_Beam_Handrail_
Testing 2 downscale_F176_VOX_arquitectura_Drawings_
Testing 3 downscale_F176_VOX_arquitectura_Facade_Handrail_
Testing 4 downscale_F176_VOX_arquitectura_Facade_Windows_
Testing 5 downscale_F176_VOX_arquitectura_Fence_Facade_Handrail_
Testing 6 downscale_F176_VOX_arquitectura_Stairs_Handrail_Steel_
Testing 7 downscale_F176_VOX_arquitectura_Windows_
Testing 8 downscale_F176_VOX_arquitectura_Windows_Brick_Facade_
Testing 9 downscale_F176_VOX_arquitectura_Windows_Facade_
Testing 10 downscale_F176_VOX_arquitectura_Windows_Fa

If there is a need to comapre metric results, modify this code to your needs.

In [ ]:
import torchvision.transforms as transforms
from basicsr.metrics import calculate_ssim, calculate_psnr, calculate_niqe
import torch
import piqa
from os.path import exists
psnr = piqa.PSNR()
ssim = piqa.ssim.SSIM()
avg_our_niqe = 0.0
avg_orig_niqe = 0.0
avg_gold_niqe = 0.0
avg_our_psnr = 0.0
avg_gold_psnr = 0.0
avg_gt_psnr = 0.0
total_files = 0
for filename in glob.glob('datasets/modernarchitecture/test/hq/*.jpg'):
    head, tail = os.path.split(filename)
    golden_result = Image.open('golden_results/downscale_'+tail[:-4]+'_out.jpg')
    our_result = Image.open('our_results/downscale_'+tail[:-4]+'_out.jpg')
    gt = Image.open(filename)
    our_result = our_result.resize((gt.size[0], gt.size[1]))
    golden_result = golden_result.resize((gt.size[0], gt.size[1]))
    transform = transforms.PILToTensor()
    golden_result = transform(golden_result)
    our_result = transform(our_result)
    gt = transform(gt)
    if(our_result.numpy().shape[0] == 3 and gt.numpy().shape == our_result.numpy().shape and gt.numpy().shape == golden_result.numpy().shape):
      avg_our_niqe += calculate_niqe(img = our_result.numpy(), crop_border=4, test_y_channel=False, input_order='CHW' ,convert_to='gray')
      avg_orig_niqe += calculate_niqe(img = gt.numpy(), crop_border=4, test_y_channel=False, input_order='CHW' ,convert_to='gray')
      avg_gold_niqe += calculate_niqe(img = golden_result.numpy(), crop_border=4, test_y_channel=False, input_order='CHW' ,convert_to='gray')
      avg_our_psnr += psnr(our_result.type(torch.float)/255, gt.type(torch.float)/255)
      avg_gold_psnr += psnr(golden_result.type(torch.float)/255, gt.type(torch.float)/255)
      avg_gt_psnr += psnr(gt.type(torch.float)/255, gt.type(torch.float)/255)
    total_files += 1
avg_our_niqe /= total_files
avg_orig_niqe /= total_files
avg_gold_niqe /= total_files
avg_our_psnr /= total_files
avg_gold_psnr /= total_files
avg_gt_psnr /= total_files
print("avg_our_niqe", avg_our_niqe, "avg_orig_niqe ", avg_orig_niqe, "avg_gold_niqe", avg_gold_niqe)
print("avg_our_psnr", avg_our_psnr, "avg_gold_psnr ", avg_gold_psnr, "avg_gt_psnr", avg_gt_psnr)


avg_our_niqe 4.36796694945366 avg_orig_niqe  3.40678806614249 avg_gold_niqe 4.437186537028811
avg_our_psnr tensor(21.8031) avg_gold_psnr  tensor(22.2831) avg_gt_psnr tensor(70.)


Here is an example of how to cut a snippet of the images to focus on specific parts of an image

In [ ]:
!mkdir patches/
from PIL import Image
import glob
ext = ".jpg"
og_filename = "F176_VOX_arquitectura_Beam_"
changed_filename = "downscale_" + og_filename + "_out"
images = []
catgories = ["our", "golden", "og"]
for category in catgories:
  if category == "og":
    image = Image.open("datasets/modernarchitecture/test/hq/" + og_filename + ".jpg")
  else:
    image = Image.open(category + "_results/" + changed_filename + ".jpg")

  width, height = image.size
  left = 0
  top = 0
  right = width / 3
  bottom = height / 6

  image = image.crop((left, top, right, bottom))
  images.append(image)

  image.save("patches/patch_" + category + "_" + og_filename + ext)